In [1]:


import sparknlp
 
spark = sparknlp.start(gpu = True) # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *

from sparknlp.common import *

import sparknlp
from sparknlp.pretrained import PretrainedPipeline
 
print("Spark NLP version", sparknlp.version())
 
print("Apache Spark version:", spark.version)



22/11/29 16:31:57 WARN Utils: Your hostname, Vader resolves to a loopback address: 127.0.1.1; using 192.168.1.94 instead (on interface enp5s0f0)
22/11/29 16:31:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/sjhjrok/anaconda3/envs/NLP/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sjhjrok/.ivy2/cache
The jars for the packages stored in: /home/sjhjrok/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp-gpu_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b3f26dc7-88ed-4b73-a3cb-0ee0f9daef93;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-gpu_2.12;4.2.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.project

22/11/29 16:31:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark NLP version 4.2.2
Apache Spark version: 3.3.1


In [2]:


from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, './eng_custom.train')
testing_data= CoNLL().readDataset(spark, './eng_custom.testa')



In [3]:
import pyspark.sql.functions as F

#training_data.select(F.explode(F.arrays_zip('token.result', 'pos.result',  'label.result')).alias("cols")) \
#.select(F.expr("cols['0']").alias("token"),
#        F.expr("cols['1']").alias("pos"),
#        F.expr("cols['2']").alias("ner_label")).show(truncate=50)


In [4]:
!mkdir ner_logs

mkdir: cannot create directory ‘ner_logs’: File exists


In [5]:


# You can use any word embeddings you want (Glove, Elmo, Bert, custom etc.)
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')
embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

nerTagger = NerDLApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(500)\
      .setLr(0.003)\
      .setBatchSize(32)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setValidationSplit(0.2)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setOutputLogsPath('ner_logs') # if not set, logs will be written to ~/annotator_logs
 #    .setGraphFolder('graphs') >> put your graph file (pb) under this folder if you are using a custom graph generated thru 4.1 NerDL-Graph.ipynb notebook
 #    .setEnableMemoryOptimizer() >> if you have a limited memory and a large conll file, you can set this True to train batch by batch 
    
ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

ner_pipeline = Pipeline(stages=[
      embeddings,
      nerTagger,
      ner_converter
 ])



glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ]glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ / ]Download done! Loading the resource.
[OK!]


In [ ]:
%%time
ner_model = ner_pipeline.fit(training_data)
ner_model.stages[1].write().overwrite().save('outputs/ner_wiki_glove100d_en')

2022-11-29 16:32:11.700037: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-29 16:32:11.700052: I external/org_tensorflow/tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-29 16:32:12.191770: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 16:32:12.192330: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open 

Training started - total epochs: 500 - lr: 0.003 - batch size: 32 - labels: 9 - chars: 22 - training examples: 125
Epoch 1/500 started, lr: 0.003, dataset size: 125
Epoch 1/500 - 0.75s - loss: 40.85604 - batches: 5
Quality on validation dataset (20.0%), validation examples = 25
time to finish evaluation: 0.19s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 77	 0	 0.11494253	 1.0	 0.20618556
Loc	 13	 18	 0	 0.41935483	 1.0	 0.59090906
Act	 8	 0	 23	 1.0	 0.2580645	 0.4102564
Neg	 0	 0	 1	 0.0	 0.0	 0.0
0	 7	 3	 58	 0.7	 0.10769231	 0.18666667
Obj	 0	 0	 14	 0.0	 0.0	 0.0
Pos	 0	 0	 2	 0.0	 0.0	 0.0
Trigger	 13	 0	 0	 1.0	 1.0	 1.0
tp: 51 fp: 98 fn: 98 labels: 8
Macro-average	 prec: 0.40428716, rec: 0.4207196, f1: 0.41233972
Micro-average	 prec: 0.34228188, rec: 0.34228188, f1: 0.34228188
Epoch 2/500 started, lr: 0.0029850747, dataset size: 125
Epoch 2/500 - 0.10s - loss: 30.963142 - batches: 5
Quality on validation dataset (20.0%), validation examples = 25
time to finish evaluation: 0.01s
la

Epoch 14/500 - 0.10s - loss: 0.2229725 - batches: 5
Quality on validation dataset (20.0%), validation examples = 25
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 31	 0	 0	 1.0	 1.0	 1.0
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 65	 0	 0	 1.0	 1.0	 1.0
Obj	 14	 0	 0	 1.0	 1.0	 1.0
Pos	 2	 0	 0	 1.0	 1.0	 1.0
Trigger	 13	 0	 0	 1.0	 1.0	 1.0
tp: 149 fp: 0 fn: 0 labels: 8
Macro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Micro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Epoch 15/500 started, lr: 0.0028037382, dataset size: 125
Epoch 15/500 - 0.10s - loss: 0.31068397 - batches: 5
Quality on validation dataset (20.0%), validation examples = 25
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 31	 0	 0	 1.0	 1.0	 1.0
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 65	 0	 0	 1.0	 1.0	 1.0
Obj	 14	 0	 0	 1.0	 1.0	 1.0
Pos	 2	 0	 0	 1.0	 1.0	 1.0
Trigger	 13	 0	 0	 1.0	 1.0	

Epoch 28/500 - 0.10s - loss: 0.05814751 - batches: 5
Quality on validation dataset (20.0%), validation examples = 25
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 31	 0	 0	 1.0	 1.0	 1.0
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 65	 0	 0	 1.0	 1.0	 1.0
Obj	 14	 0	 0	 1.0	 1.0	 1.0
Pos	 2	 0	 0	 1.0	 1.0	 1.0
Trigger	 13	 0	 0	 1.0	 1.0	 1.0
tp: 149 fp: 0 fn: 0 labels: 8
Macro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Micro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Epoch 29/500 started, lr: 0.002631579, dataset size: 125
Epoch 29/500 - 0.09s - loss: 0.104096144 - batches: 5
Quality on validation dataset (20.0%), validation examples = 25
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 31	 0	 0	 1.0	 1.0	 1.0
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 65	 0	 0	 1.0	 1.0	 1.0
Obj	 14	 0	 0	 1.0	 1.0	 1.0
Pos	 2	 0	 0	 1.0	 1.0	 1.0
Trigger	 13	 0	 0	 1.0	 1.0

Epoch 42/500 - 0.10s - loss: 0.036542263 - batches: 5
Quality on validation dataset (20.0%), validation examples = 25
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 31	 0	 0	 1.0	 1.0	 1.0
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 65	 0	 0	 1.0	 1.0	 1.0
Obj	 14	 0	 0	 1.0	 1.0	 1.0
Pos	 2	 0	 0	 1.0	 1.0	 1.0
Trigger	 13	 0	 0	 1.0	 1.0	 1.0
tp: 149 fp: 0 fn: 0 labels: 8
Macro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Micro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Epoch 43/500 started, lr: 0.0024793388, dataset size: 125
Epoch 43/500 - 0.10s - loss: 0.016874691 - batches: 5
Quality on validation dataset (20.0%), validation examples = 25
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 31	 0	 0	 1.0	 1.0	 1.0
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 65	 0	 0	 1.0	 1.0	 1.0
Obj	 14	 0	 0	 1.0	 1.0	 1.0
Pos	 2	 0	 0	 1.0	 1.0	 1.0
Trigger	 13	 0	 0	 1.0	 1

...

Epoch 196/500 - 0.10s - loss: 0.0018171615 - batches: 5
Quality on validation dataset (20.0%), validation examples = 25
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 31	 0	 0	 1.0	 1.0	 1.0
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 65	 0	 0	 1.0	 1.0	 1.0
Obj	 14	 0	 0	 1.0	 1.0	 1.0
Pos	 2	 0	 0	 1.0	 1.0	 1.0
Trigger	 13	 0	 0	 1.0	 1.0	 1.0
tp: 149 fp: 0 fn: 0 labels: 8
Macro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Micro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Epoch 197/500 started, lr: 0.0015151515, dataset size: 125
Epoch 197/500 - 0.10s - loss: 5.7656766E-4 - batches: 5
Quality on validation dataset (20.0%), validation examples = 25
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 31	 0	 0	 1.0	 1.0	 1.0
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 65	 0	 0	 1.0	 1.0	 1.0
Obj	 14	 0	 0	 1.0	 1.0	 1.0
Pos	 2	 0	 0	 1.0	 1.0	 1.0
Trigger	 13	 0	 0	 1

Epoch 210/500 - 0.10s - loss: 2.8407888E-4 - batches: 5
Quality on validation dataset (20.0%), validation examples = 25
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 31	 0	 0	 1.0	 1.0	 1.0
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 65	 0	 0	 1.0	 1.0	 1.0
Obj	 14	 0	 0	 1.0	 1.0	 1.0
Pos	 2	 0	 0	 1.0	 1.0	 1.0
Trigger	 13	 0	 0	 1.0	 1.0	 1.0
tp: 149 fp: 0 fn: 0 labels: 8
Macro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Micro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Epoch 211/500 started, lr: 0.0014634146, dataset size: 125
Epoch 211/500 - 0.10s - loss: 6.3508784E-4 - batches: 5
Quality on validation dataset (20.0%), validation examples = 25
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 31	 0	 0	 1.0	 1.0	 1.0
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 65	 0	 0	 1.0	 1.0	 1.0
Obj	 14	 0	 0	 1.0	 1.0	 1.0
Pos	 2	 0	 0	 1.0	 1.0	 1.0
Trigger	 13	 0	 0	 1

Epoch 224/500 - 0.10s - loss: 0.0013981471 - batches: 5
Quality on validation dataset (20.0%), validation examples = 25
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 31	 0	 0	 1.0	 1.0	 1.0
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 65	 0	 0	 1.0	 1.0	 1.0
Obj	 14	 0	 0	 1.0	 1.0	 1.0
Pos	 2	 0	 0	 1.0	 1.0	 1.0
Trigger	 13	 0	 0	 1.0	 1.0	 1.0
tp: 149 fp: 0 fn: 0 labels: 8
Macro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Micro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Epoch 225/500 started, lr: 0.0014150945, dataset size: 125
Epoch 225/500 - 0.10s - loss: 9.787375E-4 - batches: 5
Quality on validation dataset (20.0%), validation examples = 25
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 31	 0	 0	 1.0	 1.0	 1.0
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 65	 0	 0	 1.0	 1.0	 1.0
Obj	 14	 0	 0	 1.0	 1.0	 1.0
Pos	 2	 0	 0	 1.0	 1.0	 1.0
Trigger	 13	 0	 0	 1.

Epoch 238/500 - 0.10s - loss: 7.8790315E-4 - batches: 5
Quality on validation dataset (20.0%), validation examples = 25
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 31	 0	 0	 1.0	 1.0	 1.0
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 65	 0	 0	 1.0	 1.0	 1.0
Obj	 14	 0	 0	 1.0	 1.0	 1.0
Pos	 2	 0	 0	 1.0	 1.0	 1.0
Trigger	 13	 0	 0	 1.0	 1.0	 1.0
tp: 149 fp: 0 fn: 0 labels: 8
Macro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Micro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Epoch 239/500 started, lr: 0.0013698629, dataset size: 125
Epoch 239/500 - 0.10s - loss: 3.4506444E-4 - batches: 5
Quality on validation dataset (20.0%), validation examples = 25
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 31	 0	 0	 1.0	 1.0	 1.0
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 65	 0	 0	 1.0	 1.0	 1.0
Obj	 14	 0	 0	 1.0	 1.0	 1.0
Pos	 2	 0	 0	 1.0	 1.0	 1.0
Trigger	 13	 0	 0	 1

Epoch 252/500 - 0.10s - loss: 2.2014318E-4 - batches: 5
Quality on validation dataset (20.0%), validation examples = 25
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 31	 0	 0	 1.0	 1.0	 1.0
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 65	 0	 0	 1.0	 1.0	 1.0
Obj	 14	 0	 0	 1.0	 1.0	 1.0
Pos	 2	 0	 0	 1.0	 1.0	 1.0
Trigger	 13	 0	 0	 1.0	 1.0	 1.0
tp: 149 fp: 0 fn: 0 labels: 8
Macro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Micro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Epoch 253/500 started, lr: 0.0013274336, dataset size: 125
Epoch 253/500 - 0.10s - loss: 7.0644554E-4 - batches: 5
Quality on validation dataset (20.0%), validation examples = 25
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 31	 0	 0	 1.0	 1.0	 1.0
Neg	 1	 0	 0	 1.0	 1.0	 1.0
0	 65	 0	 0	 1.0	 1.0	 1.0
Obj	 14	 0	 0	 1.0	 1.0	 1.0
Pos	 2	 0	 0	 1.0	 1.0	 1.0
Trigger	 13	 0	 0	 1

In [ ]:

#predictions = ner_model.transform(testing_data) 
#predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
#           .select(F.expr("cols['0']").alias("token"),
#                   F.expr("cols['1']").alias("ground_truth"),
#                   F.expr("cols['2']").alias("prediction")).show(truncate=False)

In [ ]:
from sklearn.metrics import classification_report
 
#preds_df = predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
#                      .select(F.expr("cols['0']").alias("token"),
#                              F.expr("cols['1']").alias("ground_truth"),
#                              F.expr("cols['2']").alias("prediction")).toPandas()
 
#print(classification_report(preds_df['ground_truth'], preds_df['prediction']))

In [ ]:
#import shutil

#shutil.make_archive("/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en", 'zip', "/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en")

In [ ]:
text_list = [
    """Navagate to kitchen"""
]


In [ ]:
loaded_ner_model = NerDLModel.load("outputs/ner_wiki_glove100d_en")\
   .setInputCols(["sentence", "token", "embeddings"])\
   .setOutputCol("ner")

load_ner_pipeline = Pipeline(stages=[
      document_assembler,
      tokenizer,
      embeddings,
      loaded_ner_model,
      ner_converter
 ])


In [ ]:
from sparknlp.base import LightPipeline

text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = load_ner_pipeline.fit(prediction_data)
light = LightPipeline(prediction_model)
results = light.annotate(text)
print(results)
print(results['ner'])

In [ ]:
light.annotate("Navigate to kitchen")


In [ ]:
text = input("Enter Testing Text\n")
while(text != "exit"):
    results = light.annotate(text)
    print(results['ner'])
    text = input("Enter New Text\n")

In [ ]:
#empty_df = spark.createDataFrame([['']]).toDF('text')
#pipeline_model = ner_pipeline.fit(empty_df)
#df = spark.createDataFrame(pd.DataFrame({'text': text_list}))
#result = pipeline_model.transform(df)

In [ ]:




#from sparknlp_display import NerVisualizer



#result = predictions

#NerVisualizer().display(
#    result = result.collect()[0],
#    label_col = 'ner_chunk',
#    document_col = 'document'
#)




